In [1]:
import pandas as pd
news = pd.read_csv('../Chapter06/newsCorpora.csv', sep="\t", header=None, names=["ID","TITLE","URL","PUBLISHER","CATEGORY","STORY","HOSTNAME","TIMESTAMP"],index_col="ID")
news = news.query('PUBLISHER in ["Reuters","Huffington Post","Businessweek", "Contactmusic.com", "Daily Mail"]')

In [2]:
from gensim.models import KeyedVectors
w2v = KeyedVectors.load_word2vec_format('../Chapter07/GoogleNews-vectors-negative300.bin', binary=True)

unable to import 'smart_open.gcs', disabling that module


In [3]:
import spacy
nlp = spacy.load('en_core_web_sm')
def genword(title):
    for word in nlp(title): yield word.lemma_

from stemming.porter2 import stem
def wordparse(word):
    if word in w2v: return word
    if stem(word) in w2v: return stem(word)
    if word.strip('-') in w2v: return word.strip('-')
    if len(words := word.split('-'))>1: return " ".join([wordparse(word) for word in words])
    return word

In [4]:
import re
news_X = news['TITLE'].apply(lambda title:" ".join([wordparse(word) for word in genword(title)])).apply(lambda title:re.sub(r'\s{2,}',r' ',title))
news_Y = news['CATEGORY'].apply(lambda cate:'btem'.find(cate))

In [5]:
num_dic = {}
for title in news_X:
    for word in nlp(title):
        if not word.like_num: continue
        num_dic.setdefault(word.lemma_,0)
        num_dic[word.lemma_] += 1
num_set = set(map(lambda x:x[0], filter(lambda x:x[1]>10, num_dic.items()))) 

In [6]:
news_X = news_X.apply(lambda t:" ".join([w.lemma_ if not w.like_num or w in num_set else "0" for w in nlp(t)]))

In [7]:
from sklearn.model_selection import train_test_split
test_valid_X, train_X, test_valid_Y, train_Y = train_test_split(news_X, news_Y, test_size=0.8, random_state=0)
test_X, valid_X, test_Y, valid_Y = train_test_split(test_valid_X, test_valid_Y, test_size=0.5, random_state=0)

In [8]:
for mode in ["train","test","valid"]:
    with open(f"{mode}_X.feature.txt","w") as f:
        for row in eval(f"{mode}_X"): f.write(row+"\n")
    with open(f"{mode}_Y.txt","w") as f:
        for row in eval(f"{mode}_Y"): f.write(str(row)+"\n")